In [14]:
import numpy as np
import pandas as pd
import re, string, unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from bs4 import BeautifulSoup
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from textblob import TextBlob
from textblob import Word
import sys

dataset = pd.read_csv('/home/divyampramod-d/Projects/Mini_Projects/IMDB_Dataset.csv')
# tokenizer = ToktokTokenizer()
# nltk.download('stopwords')
# stopword_list = nltk.corpus.stopwords.words("english")

# #Remove the html strips from the dataset
# def strip_html(text):
#     soup = BeautifulSoup(text, "html.parser")
#     return soup.get_text()

# #Remove Square brackets within the text
# def remove_between_square_brackets(text):
#     return re.sub('\[[^]]*\]', '', text)

# #Removing any noise in the text
# def remove_noise(text):
#     text = strip_html(text)
#     text = remove_between_square_brackets(text)
#     return text

# dataset['review'] = dataset['review'].apply(remove_noise)

# #Removing any special characters in the dataset
# def remove_special_characters(text, remove_digits=True):
#     pattern = r'[^a-zA-z0-9\s]'
#     text = re.sub(pattern, '', text)
#     return text

# dataset['review'] = dataset['review'].apply(remove_special_characters)

# print(dataset['review'][0])

tokens = list(map(lambda x : set(x.split(" ")), dataset['review']))

In [15]:
vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word) > 0):
            vocab.add(word)
vocab = list(vocab)
word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))
target_dataset = list()

for label in dataset['sentiment']:
    if label == 'positive':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [34]:
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab), hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size, 1)) - 0.1

correct, total = (0, 0)
for iter in range(iterations):

    for i in range(len(input_dataset) - 1000):
        
        x, y = (input_dataset[i], target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x], axis = 0))
        layer_2 = sigmoid(np.dot(layer_1, weights_1_2))

        layer_2_delta = layer_2 - y
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)
        
        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1, layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        
        total += 1
        if (i % 10 == 9):
            progress = str(i / float(len(input_dataset)))
            sys.stdout.write('\rIter:' + str(iter) + ' Progress:' + progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()

correct, total = (0, 0)
for i in range(len(input_dataset) - 1000, len(input_dataset)):
    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x], axis = 0))
    layer_2 = sigmoid(layer_1.dot(weights_1_2))

    if (np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1

print("Test Accuracy:" + str(correct / float(total)))

439838
Iter:0 Progress:97.99% Training Accuracy:0.8393673469387755%
Iter:1 Progress:97.99% Training Accuracy:0.8735816326530612%
Test Accuracy:0.885


In [40]:
import sys, random, math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
tokens = list(map(lambda x: (x.split(" ")), dataset['review']))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x : x[0], wordcnt.most_common())))

word2index = {}
for i, word in enumerate(vocab):
    word2index[word] = i

concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

In [43]:
alpha, iterations = (0.05, 2)
hidden_size, window, negative = (50, 2, 5)

weights_0_1 = (np.random.rand(len(vocab), hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab), hidden_size) * 0

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

def similar(target = 'beautiful'):
    target_index = word2index[target]

    scores = Counter()

    for word, index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = weights_0_1 * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)

for rev_i, review in enumerate(input_dataset * iterations):
    for target_i in range(len(review)):
        target_samples = [review[target_i]] + list(concatenated[(np.random.rand(negative) * len(concatenated)).astype('int').tolist()])

        left_context = review[max(0, target_i - window) : target_i]
        right_context = review[target_i + 1 : min(len(review), target_i + window)]

        layer_1 = np.mean(weights_0_1[left_context + right_context], axis = 0)
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
        layer_2_delta = layer_2 - layer_2_target
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

        weights_0_1[left_context+right_context] -= layer_1_delta * alpha
        weights_1_2[target_samples] -= np.outer(layer_2_delta, layer_1) * alpha
        
    if(rev_i % 250 == 0):
        sys.stdout.write('\rProgresss:' + str(rev_i / float(len(input_dataset) * iterations)) + " " + str(similar('terrible')))
        sys.stdout.write('\rProgress:' + str(rev_i/float(len(input_dataset) * iterations)))
print(similar('terrible'))


(11557297,)